## MIMIC Extract

In [1]:
MIMIC_EXTRACT = '/Users/yanpavan/Desktop/Personal/TAU/MIMIC_Extract/data/all_hourly_data_group.h5'
data = "/Volumes/GoogleDrive/My Drive/TAU/Code/DrugLab/data"

In [2]:
import pickle
import pandas as pd
import os

In [3]:
med1 = pd.read_csv(os.path.join(data, "mimiciii/1.4/preprocessed", "med1_vectorized.csv"))
med2 = pd.read_csv(os.path.join(data, "mimiciii/1.4/preprocessed", "med2_vectorized.csv"))

In [8]:
file = open(MIMIC_EXTRACT, 'rb')
patients = pd.read_hdf(MIMIC_EXTRACT, key='patients')

In [9]:
vitals_labs = pd.read_hdf(MIMIC_EXTRACT, key='vitals_labs')

In [10]:
vitals_labs

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   
...                                                         ...   ...  ...   
99999      113369  246512     22                            0.0   NaN  NaN   
                              23                            0.0   NaN  NaN   
                              24                            0.0   NaN  NaN   
                              25                            0.0   NaN  NaN   
                              26                            0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   
...                                        ...  ...  ...             ...  ...   
99999      113369  246512     22           0.0  NaN  NaN             0.0  NaN   
                              23           0.0  NaN  NaN             0.0  NaN   
                              24           0.0  NaN  NaN             0.0  NaN   
                              25           0.0  NaN  NaN             0.0  NaN   
                              26           0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   
...                                     ..             ...  ...   
99999      113369  246512     22       NaN             0.0  ...   
                              23       NaN             0.0  ...   
                              24       NaN             0.0  ...   
                              25       NaN             0.0  ...   
                              26       NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   
...                                                       ...   
99999      113369  246512     22                          NaN   
                              23                          NaN   
                              24                          NaN   
                              25                     

In [61]:
import numpy as np
import pickle
import pandas as pd
meds = []
labtests = []
time_before = [np.random.randint(1, 48) for n in range(0, 6)]

meds.append('Insulin - Regular')
labtests.append('Glucose')

meds.append('Packed Red Blood Cells')
labtests.append('Hematocrit')

meds.append('Packed Red Blood Cells')
labtests.append('Hemoglobin')

meds.append('Packed Red Blood Cells')
labtests.append('RED BLOOD CELLS')

meds.append('Calcium')
labtests.append('CALCIUM, TOTAL')

meds.append('Albumin 5%')
labtests.append('Hematocrit')

# calicium
test_data = pd.DataFrame({
    'Medication': meds,
    'Lab Tests': labtests,
    'Time' : time_before
})

In [62]:
test_data

,Medication,Lab Tests,Time
0,Insulin - Regular,Glucose,9
1,Packed Red Blood Cells,Hematocrit,38
2,Packed Red Blood Cells,Hemoglobin,45
3,Packed Red Blood Cells,RED BLOOD CELLS,11
4,Calcium,"CALCIUM, TOTAL",35
5,Albumin 5%,Hematocrit,47


In [74]:
class AddMedEffect:

    def __init__(self, vitals_data, med1, dataset=None, label_path=None, admissions=None):
        self.vitals_data = vitals_data
        meds = med1
        meds['ADMITTIME'] = pd.to_datetime(meds['ADMITTIME'])
        self.meds = self._preprocess_meds_data(meds).drop(columns=["Unnamed: 0", "Unnamed: 0.2", "Unnamed: 0.1"])

        path = label_path if label_path is not None else '/Users/yanpavan/Desktop/Personal/TAU/DrugLab/data/mimic_extract/itemid_to_variable_map.csv'
        self.meds_label_map = pd.read_csv(path)
        self.meds_label_map = self.meds_label_map[['LEVEL2', 'MIMIC LABEL']]

        self.vitals_labtests = vitals_labs.columns.to_frame()['LEVEL2'].unique()
        self.vals = []
    
    def _preprocess_meds_data(self, meds):
        meds['ENDTIME'] = pd.to_datetime(meds['ENDTIME'])
        meds['time_in'] = meds['ENDTIME']-meds['ADMITTIME']
        meds['hours_in'] = meds['time_in'].apply(lambda x : x.total_seconds()//3600)
        return meds

    def editDistance(self, str1, str2):
        len1 = len(str1)
        len2 = len(str2)
    
        DP = [[0 for i in range(len1 + 1)]
                for j in range(2)]
    
        for i in range(0, len1 + 1):
            DP[0][i] = i
    
        for i in range(1, len2 + 1):
            
            for j in range(0, len1 + 1):
                if (j == 0):
                    DP[i % 2][j] = i
    
                elif(str1[j - 1] == str2[i-1]):
                    DP[i % 2][j] = DP[(i - 1) % 2][j - 1]

                else:
                    DP[i % 2][j] = (1 + min(DP[(i - 1) % 2][j],
                                        min(DP[i % 2][j - 1],
                                    DP[(i - 1) % 2][j - 1])))

        return DP[len2 % 2][len1]

    def are_same(self, str1, str2, threshold=0.25):
        str1 = str1.lower()
        str2 = str2.lower()
        m = len(str1)
        n = len(str2)

        return self.editDistance(str1, str2) / min(m,n) >= threshold

    def check_change_labtest_edit_distance(self, row):
        min_edit = 100000
        final_name = row
        for labtest in self.vitals_labtests:
            k = self.editDistance(row, labtest)
            if k<min_edit:
                min_edit = k
                final_name = labtest
        return final_name
    
    def check_change_labtest(self, row):
        if row in self.vitals_labtests:
            return row
        if row.lower() in self.vitals_labtests:
            return row.lower()
        k = self.meds_label_map
        query = k[k['MIMIC LABEL']==row]
        if query.shape[0]>0:
            return query['LEVEL2'].iloc[0].lower()
        else:
            self.check_change_labtest_edit_distance(row)

    def check_subs_with_meds(self, row, meds, sig_meds):
        adm = row.name[1]   # ['hadm_id']
        labtest_hours = row.name[3]     # ['hours_in']

        if adm in meds['HADM_ID'].to_list():
            meds = meds[meds['HADM_ID']==adm]
            # Calculate difference between relative medication time (time from start of admission) and relative time of labtest to get time from prescription
            temp = meds.apply(lambda r : (labtest_hours-r['hours_in'])>0 and (labtest_hours-r['hours_in'])==sig_meds[sig_meds['Medication']==r['LABEL']]['Time'].iloc[0], axis=1) 
            meds = meds[temp]
            return meds.shape[0]

        return 0

    def get_subs_with_meds(self, subjects_vitals, sig_meds, vitals_lab):

        all_meds = self.meds
        meds = all_meds[all_meds['HADM_ID'].isin(subjects_vitals['hadm_id'])]     # filter vital subjects
        meds = meds[meds['LABEL'].isin(sig_meds['Medication'])]    # Is a sig med
        final_subs = vitals_lab.apply(lambda row : self.check_subs_with_meds(row, meds, sig_meds) if row['count']>0 else 0, axis=1)

        return final_subs

    def add_med_effect(self, table):
        vitals_data = self.vitals_data
        labtests_vitals = vitals_data.columns.to_frame()['LEVEL2'].value_counts().keys().to_list()
        table['Lab Tests'] = table['Lab Tests'].apply(self.check_change_labtest)

        res = {}

        # count = 0
        # for labtest in labtests_vitals:
        #     if labtest not in table['Lab Tests'].to_list():
        #         count+=1
        #         continue
        
        for labtest in table['Lab Tests'].to_list():

            print(f'Labtest : {labtest}')

            if labtest in res.keys():
                continue

            sig_labtest_meds = table[table['Lab Tests']==labtest]
            sig_meds = sig_labtest_meds[['Medication', 'Time']]

            labtest_data = vitals_data[labtest]


            data_with_labtest_val = vitals_data[vitals_data[labtest]['count']>0]
            subjects_vitals = data_with_labtest_val.index.to_frame()[['subject_id', 'hadm_id', 'hours_in']]
            

            subs_with_meds = self.get_subs_with_meds(
                subjects_vitals=subjects_vitals,
                vitals_lab=vitals_data[labtest],
                sig_meds=sig_meds            
            )

            print(f'{subs_with_meds[subs_with_meds>0].shape[0]} labtest values updated for {labtest}')
            # labtest_data['sig_counts'] = subs_with_meds

            res[labtest] = {
                'labtest':labtest,
                'counts':subs_with_meds
            }
            pickle.dump(res, open('/Users/yanpavan/Desktop/Personal/TAU/DrugLab/results/mimic_extract/res_label.pkl', 'wb'))

        return res

In [75]:
k = AddMedEffect(vitals_labs, med1)

In [79]:
res = k.add_med_effect(table=test_data.copy())

Labtest : glucose
1538 labtest values updated for glucose
Labtest : hematocrit
290 labtest values updated for hematocrit
Labtest : hemoglobin
104 labtest values updated for hemoglobin
Labtest : red blood cell count
175 labtest values updated for red blood cell count
Labtest : calcium
0 labtest values updated for calcium
Labtest : hematocrit


In [ ]:
pickle.dump(res, open('/Users/yanpavan/Desktop/Personal/TAU/DrugLab/results/mimic_extract/res1.pkl', 'wb'))

In [ ]:
res = pickle.load(open('/Users/yanpavan/Desktop/Personal/TAU/DrugLab/results/mimic_extract/res1.pkl', 'rb'))

In [83]:
res

{'glucose': {'labtest': 'glucose',
  'counts': subject_id  hadm_id  icustay_id  hours_in
  3           145834   211552      0           0
                                   1           0
                                   2           0
                                   3           0
                                   4           0
                                              ..
  99999       113369   246512      22          0
                                   23          0
                                   24          0
                                   25          0
                                   26          0
  Length: 2200954, dtype: int64},
 'hematocrit': {'labtest': 'hematocrit',
  'counts': subject_id  hadm_id  icustay_id  hours_in
  3           145834   211552      0           0
                                   1           0
                                   2           0
                                   3           0
                                   4           

In [80]:
temp = res["hematocrit"]["counts"]
l = pd.DataFrame(temp)
l[l[0]>0]

,,,,0
subject_id,hadm_id,icustay_id,hours_in,
27130,122972,234314,40,1
28532,123114,288661,43,1
29075,179159,272120,42,1
29574,112343,258141,43,1
29632,128856,237834,40,1
...,...,...,...,...
97907,132043,254638,103,1
98016,166152,262920,62,1
98481,152219,209192,59,1


In [84]:
import tqdm
t_vital_labs = vitals_labs.copy()
for lab, val in tqdm.tqdm(res.items()):
    temp = val["counts"]
    l = pd.DataFrame(temp)
    with_med_effect = l[l[0]>0]
    for count, i in enumerate(with_med_effect.index):
        # if count==0:
        #     print(lab, i)
        #     print("Before: ", t_vital_labs.loc[i][lab])
            # print("B", t_vital_labs.loc[i][lab]['count'])
        t_vital_labs.loc[i, (lab, 'count')] = 0
        t_vital_labs.loc[i, (lab, 'mean')] = float('nan')
        t_vital_labs.loc[i, (lab, 'std')] = float('nan')
        # if count==0:
            # print("A", t_vital_labs.loc[i][lab]['count'])
            # print("After: ", t_vital_labs.loc[i][lab])

100%|██████████| 5/5 [00:00<00:00,  7.67it/s]


In [85]:
t_vital_labs

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   
...                                                         ...   ...  ...   
99999      113369  246512     22                            0.0   NaN  NaN   
                              23                            0.0   NaN  NaN   
                              24                            0.0   NaN  NaN   
                              25                            0.0   NaN  NaN   
                              26                            0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   
...                                        ...  ...  ...             ...  ...   
99999      113369  246512     22           0.0  NaN  NaN             0.0  NaN   
                              23           0.0  NaN  NaN             0.0  NaN   
                              24           0.0  NaN  NaN             0.0  NaN   
                              25           0.0  NaN  NaN             0.0  NaN   
                              26           0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   
...                                     ..             ...  ...   
99999      113369  246512     22       NaN             0.0  ...   
                              23       NaN             0.0  ...   
                              24       NaN             0.0  ...   
                              25       NaN             0.0  ...   
                              26       NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   
...                                                       ...   
99999      113369  246512     22                          NaN   
                              23                          NaN   
                              24                          NaN   
                              25                     

In [86]:
vitals_labs

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   
...                                                         ...   ...  ...   
99999      113369  246512     22                            0.0   NaN  NaN   
                              23                            0.0   NaN  NaN   
                              24                            0.0   NaN  NaN   
                              25                            0.0   NaN  NaN   
                              26                            0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   
...                                        ...  ...  ...             ...  ...   
99999      113369  246512     22           0.0  NaN  NaN             0.0  NaN   
                              23           0.0  NaN  NaN             0.0  NaN   
                              24           0.0  NaN  NaN             0.0  NaN   
                              25           0.0  NaN  NaN             0.0  NaN   
                              26           0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   
...                                     ..             ...  ...   
99999      113369  246512     22       NaN             0.0  ...   
                              23       NaN             0.0  ...   
                              24       NaN             0.0  ...   
                              25       NaN             0.0  ...   
                              26       NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   
...                                                       ...   
99999      113369  246512     22                          NaN   
                              23                          NaN   
                              24                          NaN   
                              25                     

In [87]:
t_vital_labs.equals(vitals_labs)

False

In [ ]:
if not os.path.isdir(os.path.join(data, "mimic_extract/med_effect")):
    os.mkdir(os.path.join(data, "mimic_extract/med_effect"))
t_vital_labs.to_csv(med1.to_csv(os.path.join(data, "mimic_extract/med_effect", "vital_labs_with_med_effect_data_leak.csv")))